In [1]:
import sys
sys.path.append("D:/share/TOSS/toss/")
import pickle


file_get = open("../graphs_dict.pkl",'rb') 
graphs_dict = pickle.load(file_get) 
file_get.close()

file_get = open("../TOSS_graphs_dict.pkl",'rb') 
TOSS_graphs_dict = pickle.load(file_get) 
file_get.close()


In [41]:
import copy
import numpy as np

file_get = open("../BVS_result.pkl",'rb') 
BVS_result = pickle.load(file_get) 
file_get.close()

file_get = open("../dismatch_TUNE.pkl",'rb') 
dismatch = pickle.load(file_get) 
file_get.close()

different_results = dismatch["OR_degree"] + dismatch["wrong_seperation"] + dismatch["tunation_candi"] 
print(len(different_results))

uncertain_BVS_graphs_dict = {}
uncertain_TOSS_graphs_dict = {}
for k in different_results:
    if k in TOSS_graphs_dict and k in BVS_result:
        uncertain_TOSS_graphs_dict[k] = TOSS_graphs_dict[k]
        temp = copy.deepcopy(TOSS_graphs_dict[k])
        temp['n']["OS"] = np.array(BVS_result[k],dtype="float32")
        uncertain_BVS_graphs_dict[k] = temp

print(len(uncertain_BVS_graphs_dict))
print(len(uncertain_TOSS_graphs_dict))

72056
72056
72056


In [42]:
uncertain_TOSS_graphs_dict['1001396.cif']['n']

,Element,EN,CN,SEN,R1,R2,R3,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,OS
0,25.0,2.20,4.0,11.28,119.0,105.0,103.0,717.280029,1509.030029,3248.500000,4941.000000,6987.0,9224.0,11500.0,18860.0,2.0
1,29.0,2.86,4.0,11.28,112.0,115.0,120.0,745.479980,1957.920044,5535.000000,5536.000000,7700.0,9938.0,13410.0,16020.0,2.0
2,14.0,2.82,8.0,11.28,116.0,107.0,102.0,786.520020,1577.130005,3231.580078,4355.520020,16091.0,19806.0,23790.0,29292.0,-2.0
3,52.0,3.14,8.0,11.28,136.0,128.0,121.0,869.299988,1795.000000,2686.000000,3610.050049,6666.0,6667.0,11980.0,13800.0,-2.0


In [43]:
uncertain_BVS_graphs_dict['1001396.cif']['n']

,Element,EN,CN,SEN,R1,R2,R3,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8,OS
0,25.0,2.20,4.0,11.28,119.0,105.0,103.0,717.280029,1509.030029,3248.500000,4941.000000,6987.0,9224.0,11500.0,18860.0,3.0
1,29.0,2.86,4.0,11.28,112.0,115.0,120.0,745.479980,1957.920044,5535.000000,5536.000000,7700.0,9938.0,13410.0,16020.0,2.0
2,14.0,2.82,8.0,11.28,116.0,107.0,102.0,786.520020,1577.130005,3231.580078,4355.520020,16091.0,19806.0,23790.0,29292.0,-4.0
3,52.0,3.14,8.0,11.28,136.0,128.0,121.0,869.299988,1795.000000,2686.000000,3610.050049,6666.0,6667.0,11980.0,13800.0,-1.0


In [44]:
file_get = open("../raw/uncertain_TOSS_graphs_dict.pkl",'wb') 
pickle.dump(uncertain_TOSS_graphs_dict, file_get) 
file_get.close()

file_get = open("../raw/uncertain_BVS_graphs_dict.pkl",'wb') 
pickle.dump(uncertain_BVS_graphs_dict, file_get) 
file_get.close()

In [45]:
sys.path.append("D:/share/TOSS/toss_GNN/")
from model_utils_pyg import *
from data_utils import *


import numpy as np
import torch
import pickle
import scipy.sparse as sp
from torch_geometric.data import Dataset as PYGDataset
from torch_geometric.loader import DataLoader as PYG_DataLoader
from torch_geometric.data import Data
from torch_geometric.data import Batch
from torch_geometric.data import HeteroData
from data_utils import refine_graphs_dict
from dataset_utils_pyg import *
from sklearn import metrics


def encode_onehot(labels):
    classes=["-4","-3","-2","-1","0","1","2","3","4","5","6","7"]
    num_classes = len(classes)
    labels = [str(int(i)) for i in labels]
    labels_onehot = np.zeros((len(labels), num_classes), dtype=np.int32)
    for i, label in enumerate(labels):
        if label in classes:
            index = classes.index(label)
            labels_onehot[i, index] = 1
    return labels_onehot


class TOSS_PYG_NC_MEF_DataSet(PYGDataset):
    def __init__(self, root, prefix=None, transform=None, pre_transform=None):
        self.prefix = prefix+"_" if prefix != None else ""
        super(TOSS_PYG_NC_MEF_DataSet, self).__init__(root, transform, pre_transform)
        self.data = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return ["%sgraphs_dict.pkl"%self.prefix]

    @property
    def processed_file_names(self):
        return ["%sNN_NC_PYG_dataset.pt"%self.prefix]

    def download(self):
        pass

    def process(self):
        # load data
        with open(self.raw_paths[0], 'rb') as f:
            graphs_dict = pickle.load(f)
        
        data_list = []
        for k, g in graphs_dict.items():
            nodes_data = g["n"]
            edges_data = g["e"]

            nodes_features = torch.from_numpy(nodes_data.iloc[:,0:-1].to_numpy())

            nodes_labels = torch.from_numpy(np.array(encode_onehot(nodes_data["OS"]), dtype = "float32"))

            edges_src = edges_data['Src']
            edges_dst = edges_data['Dst']
            bond_idx = np.arange(len(edges_data))

            EN = list(nodes_data["EN"])
            R1 = list(nodes_data["R1"])
            R2 = list(nodes_data["R2"])
            R3 = list(nodes_data["R3"])
            IP1 = list(nodes_data["IP1"])
            IP2 = list(nodes_data["IP2"])
            IP3 = list(nodes_data["IP3"])

            length_list = list(map(lambda x:edges_data['Length'][x], bond_idx))
            delta_EN_list = list(map(lambda x:abs(EN[edges_src[x]]-EN[edges_dst[x]]), bond_idx))
            sum_R1_list = list(map(lambda x:abs(R1[edges_src[x]]+R1[edges_dst[x]])*0.01,bond_idx))
            sum_R2_list = list(map(lambda x:abs(R2[edges_src[x]]+R2[edges_dst[x]])*0.01,bond_idx))
            sum_R3_list = list(map(lambda x:abs(R3[edges_src[x]]+R3[edges_dst[x]])*0.01,bond_idx))
            delta_IP1_list = list(map(lambda x:abs(IP1[edges_src[x]]-IP1[edges_dst[x]])*0.001,bond_idx))
            delta_IP2_list = list(map(lambda x:abs(IP2[edges_src[x]]-IP2[edges_dst[x]])*0.001,bond_idx))
            delta_IP3_list = list(map(lambda x:abs(IP3[edges_src[x]]-IP3[edges_dst[x]])*0.001,bond_idx))
            ratio1_list = list(map(lambda x:(edges_data['Length'][x]/abs(0.01*(R1[edges_src[x]]+R1[edges_dst[x]]))),bond_idx))
            ratio2_list = list(map(lambda x:(edges_data['Length'][x]/abs(0.01*(R2[edges_src[x]]+R2[edges_dst[x]]))),bond_idx))
            ratio3_list = list(map(lambda x:(edges_data['Length'][x]/abs(0.01*(R3[edges_src[x]]+R3[edges_dst[x]]))),bond_idx))
            one_over_l_list = list(map(lambda x:1/(edges_data['Length'][x]+1),bond_idx))
            square_l_list = list(map(lambda x:(edges_data['Length'][x])**2,bond_idx))

            edges_features = torch.from_numpy(np.hstack((np.array(length_list).reshape(len(length_list),1),
                                                             np.array(one_over_l_list).reshape(len(one_over_l_list),1),
                                                             np.array(square_l_list).reshape(len(square_l_list),1),
                                                             np.array(ratio1_list).reshape(len(ratio1_list),1),
                                                             np.array(ratio2_list).reshape(len(ratio2_list),1),
                                                             np.array(ratio3_list).reshape(len(ratio3_list),1),
                                                             np.array(delta_EN_list).reshape(len(delta_EN_list),1),
                                                             np.array(delta_IP1_list).reshape(len(delta_IP1_list),1),
                                                             np.array(delta_IP2_list).reshape(len(delta_IP2_list),1),
                                                             np.array(delta_IP3_list).reshape(len(delta_IP3_list),1),
                                                             np.array(sum_R1_list).reshape(len(sum_R1_list),1),
                                                             np.array(sum_R2_list).reshape(len(sum_R2_list),1),
                                                             np.array(sum_R3_list).reshape(len(sum_R3_list),1))).astype("float32"))
            # In PyG, edges are defined in pairs
            edge_index = torch.tensor([edges_src, edges_dst], dtype=torch.long)

            data = Data(x=nodes_features, edge_index=edge_index, edge_attr=edges_features, y=nodes_labels)
            data_list.append(data)  
            # self.data, self.slices = self.collate(data_list)

        
        # collate all your processed data and save it into a single file
        torch.save(data_list, self.processed_paths[0])
 
    def len(self):
        return len(self.data)

    def get(self, idx):
        return self.data[idx]

In [52]:
pyg_GCN_s = pyg_GCNPredictor(in_feats=15, hidden_feats=[256, 256, 256, 256], 
                             activation=None, residual=None, batchnorm=None, dropout=None, 
                             predictor_hidden_feats=64, n_tasks=12, predictor_dropout=0.3) 
pyg_GCN_s.load_state_dict(torch.load("../models/pyg_GCN_s_0609.pth"))

contrast_dict = {}

In [70]:
approach = "TOSS"  #" BVS"

dataset = TOSS_PYG_NC_MEF_DataSet(root="../", prefix="uncertain_%s"%approach)#.shuffle() 
data_loader = PYG_DataLoader(dataset, batch_size=100, shuffle=False, collate_fn=PYG_NC_collate, drop_last = False)

model = pyg_GCN_s
model.eval()

eval_matrix = measure_matrix()

with torch.no_grad():
    for batch_id, batch_data in enumerate(data_loader):
        batch_data = batch_data.to('cuda:0')
        labels = batch_data.y.to('cuda:0')

        model = model.to("cuda:0")

        outputs = model(batch_data)

        torch.cuda.empty_cache()
        masks = torch.ones(labels.shape)
        eval_matrix.update(outputs, labels, masks)
    abs_score = eval_matrix.absolute_correct_rate() *  100

all_true = np.vstack((eval_matrix.y_true))
all_pred = np.vstack((eval_matrix.y_pred))

def unencode(l):
    l = list(l)
    classes=["-4","-3","-2","-1","0","1","2","3","4","5","6","7"]
    return classes[l.index(max(l))]


ALL_PRED = list(map(unencode, all_pred))
ALL_TRUE = list(map(unencode, all_true))
NC_CM = metrics.confusion_matrix(ALL_PRED, ALL_TRUE, normalize="true")
acc = sum([1 for (x,y) in zip(ALL_PRED, ALL_TRUE) if x==y])/len(ALL_PRED)

contrast_dict["%s"%approach] = (ALL_PRED, ALL_TRUE, NC_CM, acc)

In [80]:
toss_pred, toss_true = contrast_dict["TOSS"][0], contrast_dict["TOSS"][1]
bvs_pred, bvs_true = contrast_dict["BVS"][0], contrast_dict["BVS"][1]    

t, b, s = 0, 0, 0
for i,j,k,l in zip(toss_true, toss_pred, bvs_true, bvs_pred):
    if i != k:
        if i == j:
            t+=1
        if k == l:
            b+=1
        s += 1
print(t,b,s)

print(round(t/s, 4), round(b/s, 4))

154758 147405 349635
0.4426 0.4216


In [81]:
toss_pred, toss_true = contrast_dict["TOSS"][0], contrast_dict["TOSS"][1]
bvs_pred, bvs_true = contrast_dict["BVS"][0], contrast_dict["BVS"][1]

num_nodes = len(toss_true)
diff_idx = [i for i in range(num_nodes) if toss_true[i] != bvs_true[i]]
acc_bvs = [i for i in diff_idx if bvs_true[i] == bvs_pred[i]]
acc_toss = [i for i in diff_idx if toss_true[i] == toss_pred[i]]
acc_bvs_ratio = len(acc_bvs) / len(diff_idx) if len(diff_idx) > 0 else 0
acc_toss_ratio = len(acc_toss) / len(diff_idx) if len(diff_idx) > 0 else 0

print(round(acc_toss_ratio, 4), round(acc_bvs_ratio, 4))


349635 857425
0.4426 0.4216


In [82]:
num_nodes = len(contrast_dict["TOSS"][0])

diff_idx = [i for i in range(num_nodes) if contrast_dict["TOSS"][1][i] != contrast_dict["BVS"][1][i]]
print(len(diff_idx), num_nodes)

acc_bvs = [i for i in diff_idx if contrast_dict["BVS"][0][i] == contrast_dict["BVS"][1][i]]
acc_toss = [i for i in diff_idx if contrast_dict["TOSS"][0][i] == contrast_dict["TOSS"][1][i]]

print(len(acc_bvs)/len(diff_idx), len(acc_toss)/len(diff_idx))

349635 857425
0.4215968080998756 0.44262731133896777


In [51]:
import pandas as pd
import plotly.graph_objects as go


cm_str = [["%.2f" % y for y in x] for x in NC_CM]
df_str = pd.DataFrame(cm_str)
tick_labels = [f'+{i}' if i >= 0 else str(i) for i in range(-4, 8)]

colorscale = [[0.0, '#9FDAF7'], [1.0, '#F9B3AD']]

fig = go.Figure(data=go.Heatmap(z=NC_CM, x=tick_labels, y=tick_labels,
                                text=df_str,texttemplate="%{text}",textfont={"size": 16},showscale=False,
                                colorscale=colorscale, hoverongaps=False))

fig.update_layout(width=900, height=900,xaxis=dict(title="Predicted Oxidation States", titlefont_size=24, tickfont_size=24, ticktext=tick_labels,
                             tickfont=dict(family="Times New Roman")),
                                        yaxis=dict(title="True Oxidation States", titlefont_size=24, tickfont_size=24, ticktext=tick_labels,
                             tickfont=dict(family="Times New Roman")))

for i in range(len(tick_labels) + 1):
    fig.add_shape(type="line",x0=-0.5, y0=i - 0.5, x1=len(tick_labels) - 0.5, y1=i - 0.5, line=dict(color="black", width=1))
for i in range(len(tick_labels) + 1):
    fig.add_shape(type="line",x0=i - 0.5, y0=-0.5, x1=i - 0.5, y1=len(tick_labels) - 0.5, line=dict(color="black", width=1))
fig.show()

fig.write_image(".png")

0.7612082689448056


In [ ]:
0.7526325917718751
0.7612082689448056

In [ ]:
TOSS = 0.7640467439674983
BVS = 0.7733770891108066